In [ ]:
!pip install PuLP
!pip install gekko
import pulp
from gekko import GEKKO

def solve_with_pulp_original():
    """
    Solves the original portfolio optimization problem using PuLP.
    """
    # Define the problem
    prob = pulp.LpProblem("Portfolio_Optimization_Original", pulp.LpMaximize)

    # Decision variables
    assets = ['A', 'B', 'C', 'D', 'E', 'F']
    x = pulp.LpVariable.dicts("x", assets, lowBound=0, cat='Continuous')

    # Parameters for the optimistic scenario
    net_returns = {'A': 0.069, 'B': 0.049, 'C': 0.088, 'D': 0.039, 'E': 0.058, 'F': 0.044}
    upper_bounds = {'A': 0.40, 'B': 0.35, 'C': 0.30, 'D': 0.25, 'E': 0.30, 'F': 0.20}

    # Objective function
    prob += pulp.lpSum([net_returns[i] * x[i] for i in assets]), "Total_Net_Return"

    # Constraints
    prob += pulp.lpSum([x[i] for i in assets]) == 1, "Total_Investment"
    for i in assets:
        prob += x[i] <= upper_bounds[i], f"Upper_Bound_{i}"

    # Solve the problem
    prob.solve()

    # Print results
    print("--- PuLP: Solução do Problema Original ---")
    print(f"Status: {pulp.LpStatus[prob.status]}")
    print("Alocação Ótima:")
    for i in assets:
        print(f"  Ativo {i}: {x[i].varValue * 100:.2f}%")
    print(f"Retorno Máximo Esperado: {pulp.value(prob.objective):.5f}\n")

def solve_with_pulp_cash_reserve():
    """
    Solves the portfolio optimization problem with a cash reserve using PuLP.
    """
    # Define the problem
    prob = pulp.LpProblem("Portfolio_Optimization_Cash_Reserve", pulp.LpMaximize)

    # Decision variables
    assets = ['A', 'B', 'C', 'D', 'E', 'F']
    x = pulp.LpVariable.dicts("x", assets, lowBound=0, cat='Continuous')

    # Parameters
    net_returns = {'A': 0.069, 'B': 0.049, 'C': 0.088, 'D': 0.039, 'E': 0.058, 'F': 0.044}
    upper_bounds = {'A': 0.40, 'B': 0.35, 'C': 0.30, 'D': 0.25, 'E': 0.30, 'F': 0.20}

    # Objective function
    prob += pulp.lpSum([net_returns[i] * x[i] for i in assets]), "Total_Net_Return"

    # Constraints
    prob += pulp.lpSum([x[i] for i in assets]) <= 0.95, "Max_Investment_95_Percent"
    for i in assets:
        prob += x[i] <= upper_bounds[i], f"Upper_Bound_{i}"

    # Solve the problem
    prob.solve()

    # Print results
    print("--- PuLP: Solução com Reserva de Caixa ---")
    print(f"Status: {pulp.LpStatus[prob.status]}")
    total_invested = sum(v.varValue for v in x.values())
    print("Alocação Ótima:")
    for i in assets:
        print(f"  Ativo {i}: {x[i].varValue * 100:.2f}%")
    print(f"  Reserva de Caixa: {(1 - total_invested) * 100:.2f}%")
    print(f"Retorno Máximo Esperado: {pulp.value(prob.objective):.5f}\n")


def solve_with_gekko_original():
    """
    Solves the original portfolio optimization problem using Gekko.
    """
    m = GEKKO(remote=False)
    assets = ['A', 'B', 'C', 'D', 'E', 'F']

    # Variables
    x = [m.Var(value=0.1, lb=0) for _ in assets]

    # Parameters
    net_returns = {'A': 0.069, 'B': 0.049, 'C': 0.088, 'D': 0.039, 'E': 0.058, 'F': 0.044}
    upper_bounds = {'A': 0.40, 'B': 0.35, 'C': 0.30, 'D': 0.25, 'E': 0.30, 'F': 0.20}

    for i, asset_name in enumerate(assets):
        x[i].upper = upper_bounds[asset_name]

    # Constraint
    m.Equation(sum(x) == 1)

    # Objective function
    m.Maximize(sum(net_returns[assets[i]] * x[i] for i in range(len(assets))))

    # Solve
    m.solve(disp=False)

    # Print results
    print("--- Gekko: Solução do Problema Original ---")
    print("Alocação Ótima:")
    for i, asset_name in enumerate(assets):
        print(f"  Ativo {asset_name}: {x[i].value[0] * 100:.2f}%")
    print(f"Retorno Máximo Esperado: {-m.options.objfcnval:.5f}\n")


def solve_with_gekko_cash_reserve():
    """
    Solves the portfolio optimization problem with a cash reserve using Gekko.
    """
    m = GEKKO(remote=False)
    assets = ['A', 'B', 'C', 'D', 'E', 'F']

    # Variables
    x = [m.Var(value=0.1, lb=0) for _ in assets]

    # Parameters
    net_returns = {'A': 0.069, 'B': 0.049, 'C': 0.088, 'D': 0.039, 'E': 0.058, 'F': 0.044}
    upper_bounds = {'A': 0.40, 'B': 0.35, 'C': 0.30, 'D': 0.25, 'E': 0.30, 'F': 0.20}

    for i, asset_name in enumerate(assets):
        x[i].upper = upper_bounds[asset_name]

    # Constraint
    m.Equation(sum(x) <= 0.95)

    # Objective
    m.Maximize(sum(net_returns[assets[i]] * x[i] for i in range(len(assets))))

    # Solve
    m.solve(disp=False)

    # Print results
    print("--- Gekko: Solução com Reserva de Caixa ---")
    total_invested = sum(xi.value[0] for xi in x)
    print("Alocação Ótima:")
    for i, asset_name in enumerate(assets):
        print(f"  Ativo {asset_name}: {x[i].value[0] * 100:.2f}%")
    print(f"  Reserva de Caixa: {(1 - total_invested) * 100:.2f}%")
    print(f"Retorno Máximo Esperado: {-m.options.objfcnval:.5f}\n")


if __name__ == '__main__':
    # --- Part 1: Original Problem ---
    solve_with_pulp_original()
    solve_with_gekko_original()

    # --- Part 2: Problem with Cash Reserve ---
    solve_with_pulp_cash_reserve()
    solve_with_gekko_cash_reserve()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 43.5 MB/s eta 0:00:00
--- PuLP: Solução do Problema Original ---
Status: Optimal
Alocação Ótima:
  Ativo A: 40.00%
  Ativo B: 0.00%
  Ativo C: 30.00%
  Ativo D: 0.00%
  Ativo E: 30.00%
  Ativo F: 0.00%
Retorno Máximo Esperado: 0.07140

--- Gekko: Solução do Problema Original ---
Alocação Ótima:
  Ativo A: 40.00%
  Ativo B: 0.00%
  Ativo C: 30.00%
  Ativo D: 0.00%
  Ativo E: 30.00%
  Ativo F: 0.00%
Retorno Máximo Esperado: 0.07140

--- PuLP: Solução com Reserva de Caixa ---
Status: Optimal
Alocação Ótima:
  Ativo A: 40.00%
  Ativo B: 0.00%
  Ativo C: 30.00%
  Ativo D: 0.00%
  Ativo E: 25.00%
  Ativo F: 0.00%
  Reserva de Caixa: 5.00%
Retorno Máximo Esperado: 0.06850

--- Gekko: Solução com Reserva de Caixa ---
Alocação Ótima:
  Ativo A: 40.00%
  Ativo B: 0.00%
  Ativo C: 30.00%
  Ativo D: 0.00%
  Ativo E: 25.00%
  Ativo F: 0.00%
  Reserva de Caixa: 5.0

In [1]:
import pulp

# Definir o problema
prob = pulp.LpProblem("Sistema_Linear_Intervalar", pulp.LpMinimize)

# Definir as variáveis
x1 = pulp.LpVariable("x1", lowBound=0)  # x1 >= 0
x2 = pulp.LpVariable("x2", lowBound=0)  # x2 >= 0

# Função objetivo (não há necessidade de objetivo, apenas para representar o modelo)
prob += 0  # Nenhum custo associado

# Restrições
prob += 2*x1 + 0*x2 >= 0  # Restrição de [2, 3]x1 + [0, 1]x2 = [0, 120]
prob += 1*x1 + 2*x2 >= 60  # Restrição de [1, 2]x1 + [2, 3]x2 = [60, 240]

# Resolver o problema
prob.solve()

# Imprimir os resultados
print("Status:", pulp.LpStatus[prob.status])
print("x1 =", pulp.value(x1))
print("x2 =", pulp.value(x2))


ModuleNotFoundError: No module named 'pulp'

In [2]:
from gekko import GEKKO

# Criando o modelo
m = GEKKO(remote=False)

# Variáveis
x1 = m.Var(value=0, name="x1", lb=0)
x2 = m.Var(value=0, name="x2", lb=0)

# Definir as equações
m.Equation(2*x1 + 0*x2 == 0)  # Representando [2, 3]x1 + [0, 1]x2 = [0, 120]
m.Equation(1*x1 + 2*x2 == 60)  # Representando [1, 2]x1 + [2, 3]x2 = [60, 240]

# Resolver o modelo
m.solve(disp=True)

# Exibir os resultados
print("x1 =", x1.value[0])
print("x2 =", x2.value[0])


ModuleNotFoundError: No module named 'gekko'

In [4]:

!pip install pulp
!pip install gekko

# Importando as bibliotecas necessárias
import pulp
from gekko import GEKKO

# --- Parte 1: Solução com PuLP ---

# Definir o problema de otimização com PuLP
prob = pulp.LpProblem("Sistema_Linear_Intervalar_PuLP", pulp.LpMinimize)

# Definir as variáveis (não-negativas)
x1 = pulp.LpVariable("x1", lowBound=0)  # x1 >= 0
x2 = pulp.LpVariable("x2", lowBound=0)  # x2 >= 0

# Função objetivo (não há necessidade de objetivo, apenas para representar o modelo)
prob += 0  # Nenhum custo associado

# Restrições (representando os sistemas lineares)
prob += 2 * x1 + 0 * x2 >= 0  # Restrição de [2, 3]x1 + [0, 1]x2 = [0, 120]
prob += 1 * x1 + 2 * x2 >= 60  # Restrição de [1, 2]x1 + [2, 3]x2 = [60, 240]

# Resolver o problema com PuLP
prob.solve()

# Exibir o status da solução e os valores das variáveis usando PuLP
print("--- Resultado com PuLP ---")
print("Status:", pulp.LpStatus[prob.status])
print("x1 =", pulp.value(x1))
print("x2 =", pulp.value(x2))

# --- Parte 2: Solução com Gekko ---

# Criando o modelo com Gekko
m = GEKKO(remote=False)

# Definir as variáveis (não-negativas)
x1_gk = m.Var(value=0, name="x1", lb=0)
x2_gk = m.Var(value=0, name="x2", lb=0)

# Definir as equações (restrições)
m.Equation(2 * x1_gk + 0 * x2_gk == 0)  # Representando [2, 3]x1 + [0, 1]x2 = [0, 120]
m.Equation(1 * x1_gk + 2 * x2_gk == 60)  # Representando [1, 2]x1 + [2, 3]x2 = [60, 240]

# Resolver o modelo com Gekko
m.solve(disp=True)

# Exibir os resultados de Gekko
print("\n--- Resultado com Gekko ---")
print("x1 =", x1_gk.value[0])
print("x2 =", x2_gk.value[0])


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 94.0 MB/s eta 0:00:00
--- Resultado com PuLP ---
Status: Optimal
x1 = 60.0
x2 = 0.0
 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            2
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:              2
 Number of total equations: -            2
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 solver            3  not supported
 using default solver: APOPT
 ------------------------